In [1]:
import tifffile
import pandas

from griottes.analyse import cell_property_extraction
from griottes.graphmaker import graph_generation_func
from griottes.graphplotter import graph_plot
import griottes

# Load image and mask of nuclei

From the image we can extract the single cell information. For the analysis, it is necessary that the channels be structured as $Z \times X \times Y \times C$, where $C$ is the fluorescence channels in a multi-channel image.

The file `total_image.tiff` can be downloaded from by executing the box below.

In [5]:
import urllib3
import shutil

url = 'https://github.com/BaroudLab/Griottes/releases/download/v1.0-alpha/spheroid_image.tiff'
filename = 'spheroid_image.tiff'

c = urllib3.PoolManager()

with c.request('GET',url, preload_content=False) as resp, open(filename, 'wb') as out_file:
    shutil.copyfileobj(resp, out_file)

resp.release_conn()

# Extract single-cell information from the image

In [6]:
# THIS ANDREY IS WHERE I NEED THE BINNED 3D SPHEROID
# IMAGE.

spheroid_image = tifffile.imread('spheroid_image.tiff')
spheroid_image = total_image[200:400, 300:500, ...]

prop = cell_property_extraction.get_cell_properties(
    spheroid_image,
    mask_channel = 3,
    analyze_fluo_channels = True,
    fluo_channel_analysis_method = 'local_voronoi',
    cell_geometry_properties = True,
    radius = 35,
    labeled_voronoi_tesselation = False,
    percentile = 95,
    min_area = 400
    )

FileNotFoundError: [Errno 2] No such file or directory: '/home/pub/Documents/Code/spheroid-graphs/example_notebooks/total_image.tiff'

From the extracted positions of the cell centers, it is possible to generate a network representation of the 3D spheroid. For practical purposes, we have saved the dataframe containing the previously extracted information which is re-used below.

For illustration purposes we decide to attribute different cell types depending on the fluorescence measured within the mask. Any cell which mean_intensity_1 is above the median is considered CD146+, otherwise it is CD146-.

In [ ]:
prop['cell_type'] = (prop.mean_intensity_1 > prop.mean_intensity_1.median()).astype(int)
prop.index = np.arange(len(prop))

legend_list = ['CD146-', 'CD146+']
prop['legend'] = [legend_list[prop.loc[i, 'cell_type']] for i in range(len(prop))]

color_list = [plt.cm.Set3(i) for i in range(2)]
prop['color'] = [color_list[prop.loc[i, 'cell_type']] for i in range(len(prop))]


# Delaunay-based network construction

In [ ]:
descriptors = ['x', 'y', 'legend', 'color', 'cell_type']

G = graph_generation_func.generate_delaunay_graph(prop, 
                                                 descriptors = descriptors, 
                                                 dCells = 100,
                                                 flat_image = True)

From the graph G it is possible to plot a visual representation of the network. Griottes contains several specific plotting functions adapted for the network representation of tisssues. These functions are called through the graph_plot module. Here we call a specific function for 3D plots.

In [ ]:
graph_plot.network_plot_3D(G, 
                figsize = (7, 7),
                alpha_line=0.6,
                scatterpoint_size=10,
                legend=True,
                legend_fontsize = 18,
                theta = -0,
                psi = -0,
                xlim = (prop.x.min() - 5, prop.x.max() + 5),
                ylim = (prop.y.min() - 5, prop.y.max() + 5),
                zlim = (prop.z.min() - 5, prop.z.max() + 5))